In [6]:
import os
import re
import time
import glob
import numpy as np
import pandas as pd
import lxml
from requests import get
from bs4 import BeautifulSoup
from basketball_reference_scraper.teams import get_roster_stats as grs
from basketball_reference_scraper.utils import remove_accents
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from constants import DATA_DIR #TEAMS, SECONDS_SLEEP

In [9]:
class DataScraper():
    def get_roster_stats(self, team, season):
        df = grs(team, season)
        return df
    
    # some players were traded midseason and therefore appear more than once, let's merge them
    def merge_duplicates(self, df):
        duplicate = df.pivot_table(index=['PLAYER'], aggfunc='size')
        duplicate = duplicate[duplicate != 1]
        for player in duplicate.index:
            df1 = df.loc[df['PLAYER'] == player].fillna(0)
            df1 = df1.apply(pd.to_numeric, errors='ignore')
            # create merged row
            merged = df1.iloc[0]
            total_games = df1['G'].sum()
            merged['G'] = total_games
            merged['GS'] = df1['GS'].sum()
            weighted_stats = pd.DataFrame()
            
            for index, row in df1._get_numeric_data().iterrows():
                weight = row['G']/total_games #calculate weight
                new_row = row*weight #muliply weight to get new values
                weighted_stats = weighted_stats.append(new_row) #add to df
            
            weighted_stats = weighted_stats.sum(axis=0, skipna=True)
            merged_stats = ['MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA', 
                            'ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
            
            for stat in merged_stats:
                merged[stat] = weighted_stats[stat]
            
            # calulate % stats
            merged['FG%'] = merged['FG']/merged['FGA']
            merged['3P%'] = merged['3P']/merged['3PA']
            merged['2P%'] = merged['2P']/merged['2PA']
            merged['eFG%'] = (merged['FG']+0.5*merged['3P'])/merged['FGA']
            merged['FT%'] = merged['FT']/merged['FTA']
            merged=merged.fillna(0)
            
            df = df[df.PLAYER != player]
            df = df.append(merged)
           
        return df
        

In [10]:
if not os.path.exists(os.path.join(DATA_DIR,'PlayerStats')):
    os.mkdir(os.path.join(DATA_DIR,'PlayerStats'))

SEASONS = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
TEAMS  = ['ATL','BRK','BOS','CHI','CHO','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM',
          'MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS']

scraper = DataScraper()
for season in tqdm(SEASONS):
    df = pd.DataFrame()
    for team in tqdm(TEAMS):
        df1 = scraper.get_roster_stats(team, season)
        df = df.append(df1)
    df = scraper.merge_duplicates(df)
    df['PLAYER'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df.fillna(0, inplace=True)
    df.to_csv(DATA_DIR+'/PlayerStats/'+f'{season-1}-{str(season)[2:]}.csv', index=False)

<ipython-input-9-a31a5f9d3ece>:34: RuntimeWarning: invalid value encountered in double_scalars
  merged['3P%'] = merged['3P']/merged['3PA']
<ipython-input-9-a31a5f9d3ece>:37: RuntimeWarning: invalid value encountered in double_scalars
  merged['FT%'] = merged['FT']/merged['FTA']


<ipython-input-9-a31a5f9d3ece>:35: RuntimeWarning: invalid value encountered in double_scalars
  merged['2P%'] = merged['2P']/merged['2PA']
